In [2]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd
import numpy as np
from beakerx import *
from beakerx.object import beakerx
import seaborn as sns
import matplotlib.pyplot as plt
import scipy as sci
from copy import copy
from IPython.display import HTML
from multiprocessing import Pool, cpu_count

# Import the data

In [3]:
crsp_raw = pd.read_csv('../Data/crsp.txt', sep = '\t', low_memory = False, nrows = 10000)

In [11]:
compustat_raw = pd.read_csv('./Data/compustat-merged.txt', sep = '\t', low_memory = False, nrows = 10000)

In [5]:
def clean_data(df, type_dict):
    print('Cleaning date variables:')
    for v in type_dict['date_vars']:
        print(v)
        df[v] = pd.to_datetime(df[v], format = '%Y/%m/%d', errors = 'coerce', cache = True).dt.tz_localize(None) + MonthEnd(0)
        
    print('Cleaning numeric variables:')
    for v in type_dict['float_vars']:
        print(v)
        df[v] = pd.to_numeric(df[v], errors = 'coerce')
    
    print('Cleaning integer variables:')
    for v in type_dict['int_vars']:
        print(v)
        df[v] = pd.to_numeric(df[v], downcast = 'signed', errors = 'coerce')
        
    print('Final data types:')
    print(df.dtypes)
    
    return df

## Cleaning Up CRSP

In [6]:
crsp_datatypes = {'date_vars': ['date'],
                 'float_vars': ['BIDLO', 'ASKHI', 'PRC', 'RET', 'DLRET', 'BID', 'ASK', 'RETX', 'CFACPR', 'CFACSHR'],
                 'int_vars': ['PERMNO', 'PERMCO', 'HSICCD', 'CUSIP', 'SHROUT', 'VOL']}
crsp = clean_data(copy(crsp_raw), crsp_datatypes)

Cleaning date variables:
date
Cleaning numeric variables:
BIDLO
ASKHI
PRC
RET
DLRET
BID
ASK
RETX
CFACPR
CFACSHR
Cleaning integer variables:
PERMNO
PERMCO
HSICCD
CUSIP
SHROUT
VOL
Final data types:
PERMNO              int32
date       datetime64[ns]
TICKER             object
COMNAM             object
SHRCLS             object
PERMCO               int8
HSICCD              int16
CUSIP             float64
DLSTCD            float64
NWPERM            float64
DLPRC             float64
DLRET             float64
BIDLO             float64
ASKHI             float64
PRC               float64
VOL               float64
RET               float64
BID               float64
ASK               float64
SHROUT            float64
CFACPR            float64
CFACSHR           float64
RETX              float64
dtype: object


In [8]:
# Choose the right variables
crsp_names = {'RET': 'Return', 
              'SHROUT': 'Shares Outstanding on Trading Day', 
              'COMNAM': 'Company Name',
              'TICKER': 'Ticker',
              'date': 'datadate', 
              'PERMNO': 'Permno', 
              'PERMCO': 'Permco',
              'PRC': 'Price with Flag',
              'BID': 'Bid',
              'ASK': 'Ask',
              'VOL': 'Volume on Trading Day',
              'SHRCLS': 'Share Class',
              'CFACPR': 'Price Adjustment Factor',
              'CFACSHR': 'Share Adjustment Factor'}

crsp = crsp.rename(index = str, columns = crsp_names)
crsp = crsp[list(crsp_names.values())]

In [9]:
# Make a few more useful variables
crsp['Imputed Price'] = (crsp['Price with Flag'] < 0)
crsp['Price on Trading Day'] = np.abs(crsp['Price with Flag'])
crsp['Price'] = crsp['Price on Trading Day'] / crsp['Price Adjustment Factor']
crsp['Shares Outstanding'] = crsp['Shares Outstanding on Trading Day'] * crsp['Share Adjustment Factor']
crsp['Volume'] = crsp['Volume on Trading Day'] * crsp['Share Adjustment Factor']
crsp['Market Cap (Billions, CRSP)'] = crsp['Shares Outstanding'] * crsp['Price'] / 1e6

In [10]:
# Drop a few variables
crsp = crsp.drop(['Price on Trading Day', 'Shares Outstanding on Trading Day', 'Volume on Trading Day'], axis = 1)

In [17]:
crsp.head()

In [17]:
# Check for unique identifier

def check_unique(dataframe, identifier_list):
    unique_identifier = dataframe.groupby(by = identifier_list).count()
    return unique_identifier.shape[0] == dataframe.shape[0]

print(check_unique(crsp, ['Permco', 'datadate']))

False


In [18]:
crsp = crsp.reset_index().set_index(['Permco', 'datadate'])

In [19]:
AGG_VAR_TYPES = {'First': ['Company Name', 'Ticker', 'Price', 'Bid', 'Ask'],
             'Add': ['Market Cap (Billions, CRSP)'],
             'Weighted Sum': ['Volume', 'Return']}
ALL_CRSP_VAR = [item for sublist in AGG_VAR_TYPES.values() for item in sublist]

def agg_share_classes(company_on_date):
    
    if ('Loop Number' in company_on_date.columns) and company_on_date['Loop Number'].values[0] % 1000 == 0:
        print(company_on_date.index[0])
    
    # If only one share class
    if company_on_date.shape[1] <= 1:
        return company_on_date[ALL_CRSP_VAR]
    
    print('Going for remaing cases')
    # Then for the remainign cases
    company_on_date['Market Cap (Billions, CRSP)'].fillna(0)
    largest_market_cap = company_on_date['Market Cap (Billions, CRSP)'].max()
    largest_class = (company_on_date['Market Cap (Billions, CRSP)'] == largest_market_cap)
    
    # If there is no valid market cap, just pick the first record arbitrarily
    if largest_market_cap > 0:
        find_valid = np.where(largest_class)[0]
        if find_valid.shape[0] > 1:
            largest_class[find_valid[0] + 1:] = False
            largest_class[find_valid[0]] = True
    else:
        largest_class[0] = True
        largest_class[1:] = False
    
    print(largest_class)
    assert(np.where(largest_class)[0].shape[0] == 1)
        
    new_frame = {}
    for v in AGG_VAR_TYPES['First']:
        new_frame[v] = company_on_date.loc[largest_class, v]

    for v in AGG_VAR_TYPES['Add']:
        new_frame[v] = company_on_date[v].fillna(0).sum()

    for v in AGG_VAR_TYPES['Weighted Sum']:
        valid = ~company_on_date[v].isnull()
        denominator = company_on_date.loc[valid, 'Market Cap (Billions, CRSP)'].sum()
        
        if denominator > 0:
            new_frame[v] = (company_on_date.loc[valid, v] * company_on_date.loc[valid, 'Market Cap (Billions, CRSP)']).sum() / denominator
        else:
            new_frame[v] = company_on_date[v][0]
        
    return pd.DataFrame.from_dict(new_frame)

In [22]:
# First, pull every permco-date that isn't paired
crsp_counts = crsp['Permno'].groupby(by = ['Permco', 'datadate']).count()
crsp_counts.name = 'Count'
crsp_counts = crsp_counts[crsp_counts > 1]
crsp_counts = crsp_counts.to_frame()

In [23]:
crsp_merge = crsp_counts.join(crsp, how = 'outer')

In [24]:
problem_children = crsp_merge.loc[~pd.isnull(crsp_merge['Count']), ALL_CRSP_VAR]
good_children = crsp_merge.loc[pd.isnull(crsp_merge['Count']), ALL_CRSP_VAR]

In [25]:
problem_children['Loop Number'] = list(range(problem_children.shape[0]))

In [26]:
merged_problems = problem_children.groupby(['Permco', 'datadate']).apply(agg_share_classes)

(32, Timestamp('1983-06-30 00:00:00'))
Going for remaing cases
Permco  datadate  
32      1983-06-30     True
        1983-06-30    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1983-07-31    False
        1983-07-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1983-08-31     True
        1983-08-31    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1983-09-30     True
        1983-09-30    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1983-10-31     True
        1983-10-31    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1983-11-30     True
        1983-11-30    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1983-12-31    False
        1983-1

Going for remaing cases
Permco  datadate  
32      1988-05-31     True
        1988-05-31    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1988-06-30    False
        1988-06-30     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1988-07-31    False
        1988-07-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1988-08-31     True
        1988-08-31    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1988-09-30    False
        1988-09-30     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1988-10-31    False
        1988-10-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1988-11-30    False
        1988-11-30     True
Name: Market Cap (Billion

Going for remaing cases
Permco  datadate  
32      1994-02-28    False
        1994-02-28     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1994-03-31    False
        1994-03-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1994-04-30    False
        1994-04-30     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1994-05-31    False
        1994-05-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1994-06-30    False
        1994-06-30     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1994-07-31    False
        1994-07-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
32      1994-08-31    False
        1994-08-31     True
Name: Market Cap (Billion

Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2009-12-31     True
        2009-12-31    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2010-01-31     True
        2010-01-31    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2010-02-28     True
        2010-02-28    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2010-03-31     True
        2010-03-31    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2010-04-30     True
        2010-04-30    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2010-05-31     True
        2010-05-31    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2010-06-30     True
      

Going for remaing cases
Permco  datadate  
33      2015-07-31     True
        2015-07-31    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2015-08-31     True
        2015-08-31    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2015-09-30     True
        2015-09-30    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2015-10-31     True
        2015-10-31    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2015-11-30     True
        2015-11-30    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2015-12-31     True
        2015-12-31    False
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
33      2016-01-31     True
        2016-01-31    False
Name: Market Cap (Billion

Permco  datadate  
47      1990-02-28    False
        1990-02-28     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1990-03-31    False
        1990-03-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1990-04-30    False
        1990-04-30     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1990-05-31    False
        1990-05-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1990-06-30    False
        1990-06-30     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1990-07-31    False
        1990-07-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1990-08-31    False
        1990-08-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Go

Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1995-02-28    False
        1995-02-28     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1995-03-31    False
        1995-03-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1995-04-30    False
        1995-04-30     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1995-05-31    False
        1995-05-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1995-06-30    False
        1995-06-30     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1995-07-31    False
        1995-07-31     True
Name: Market Cap (Billions, CRSP), dtype: bool
Going for remaing cases
Permco  datadate  
47      1995-08-31    False
      

In [28]:
crsp_merge = pd.concat([good_children[ALL_CRSP_VAR], merged_problems[ALL_CRSP_VAR]])

In [29]:
apple = crsp.xs(7, level = 'Permco', drop_level = False)

In [30]:
apple = apple.loc[~pd.isnull(apple['Return'])]
times = apple.index.get_level_values(1)

In [31]:
x = times.shift(1, freq = 'M') - times

In [32]:
x.max().total_seconds() < 32 * 24 * 60 * 60

True

In [33]:
def continuous_returns(company_dataframe):
    """
    Detects if a company has continuous return data. Returns "True" if the returns are continuous. False otherwise
    :param company_dataframe -- a dataframe from a groupby object, indexed on Permco
    :returns True or False, depending on whether the return sequence is continuous
    """
    print(company_dataframe.head())
    times = company_dataframe.index.get_level_values(1)
    diffs = times.shift(1, freq = 'M') - times
    return (diffs.max().total_seconds() < 32 * 24 * 60 * 60)
    

In [34]:
test_returns = crsp_merge.iloc[0:1000, :]

In [35]:
test_returns.groupby(by = ['Permco']).apply(continuous_returns)

                                   Company Name Ticker   Price  Bid  Ask  \
Permco datadate                                                            
2      1972-11-30                           NaN    NaN     NaN  NaN  NaN   
       1972-12-31  ANGLO AMERICAN GOLD INVT LTD  AAGIY  2.3000  NaN  NaN   
       1973-01-31  ANGLO AMERICAN GOLD INVT LTD  AAGIY  2.7125  NaN  NaN   
       1973-02-28  ANGLO AMERICAN GOLD INVT LTD  AAGIY  3.0625  NaN  NaN   
       1973-03-31  ANGLO AMERICAN GOLD INVT LTD  AAGIY  3.5125  NaN  NaN   

                   Market Cap (Billions, CRSP)  Volume    Return  
Permco datadate                                                   
2      1972-11-30                          NaN     NaN       NaN  
       1972-12-31                     0.504896     NaN       NaN  
       1973-01-31                     0.595448     NaN  0.179348  
       1973-02-28                     0.672280     NaN  0.157272  
       1973-03-31                     0.771064     NaN  0.146939 

Permco
2    True
3    True
4    True
5    True
dtype: bool

In [36]:
# Build a visualizer

def simple_plot(dataframe, variable, plot_title):
    plot = TimePlot(title = plot_title, legendLayout=LegendLayout.HORIZONTAL,\
                          legendPosition=LegendPosition(position=LegendPosition.Position.RIGHT),\
                        logY = True,
                        yLogBase = np.exp(1))
    plot.add(Line(displayName = variable, \
                  x = dataframe['datadate'],\
                  y = dataframe[variable]))
    return plot

def price_and_market_cap(dataframe, permco, company_name):
    
    target_view = dataframe[dataframe['Permco'] == permco]
    
    cplot = CombinedPlot(xLabel = 'Date', initWidth = 600, initHeight = 1000)
    cplot.add(simple_plot(target_view, 'Price', 'Price of ' + company_name), 6)
    cplot.add(simple_plot(target_view, 'Market Cap (Billions, CRSP)', 'Market Cap'), 6)
    cplot.add(simple_plot(target_view, 'Volume', 'Volume'), 6)
    
    return cplot

In [37]:
def find_permcos(dataframe, ticker):
    right_ticker = dataframe.loc[dataframe['Ticker'] == ticker]
    return right_ticker.groupby(by = ['Permco']).first()
    

In [38]:
find_permcos(crsp_merge, 'AMZN')

In [39]:
price_and_market_cap(crsp_merge, 15473, 'Amazon')

KeyError: 'Permco'

In [ ]:
# Output the data
store = pd.HDFStore('Output/data.h5')
store.put('crsp_only', crsp_merge)

## Cleaning up Compustat

In [40]:
# Define the data types
compustat_datatypes = {'date_vars': ['datadate'],
                 'float_vars': ['cogsq', 'cshopq', 'cshoq', 'dpq', 'oiadpq', 'oibdpq', 'prcraq', 'saleq', 'txpq', 'xintq', 'xsgaq', 'aqcy', 'capxy', 'dvy', 'fincfy', 'oancfy', 'prccq', 'seqq', 'ceqq', 'pstkrq', 'atq', 'ltq', 'cheq'],
                 'int_vars': ['GVKEY', 'LPERMNO', 'LPERMCO', 'fyearq', 'fqtr', 'cusip', 'exchg', 'cik', 'naics']}
compustat = clean_data(copy(compustat_raw), compustat_datatypes)

Cleaning date variables:
datadate
Cleaning numeric variables:
cogsq
cshopq
cshoq
dpq
oiadpq
oibdpq
prcraq
saleq
txpq
xintq
xsgaq
aqcy
capxy
dvy
fincfy
oancfy
prccq
seqq
ceqq
pstkrq
atq
ltq
cheq
Cleaning integer variables:
GVKEY
LPERMNO
LPERMCO
fyearq
fqtr
cusip
exchg
cik
naics
Final data types:
GVKEY                int16
LPERMNO              int32
LPERMCO              int32
datadate    datetime64[ns]
fyearq               int16
fqtr                  int8
indfmt              object
consol              object
popsrc              object
datafmt             object
tic                 object
cusip              float64
conm                object
curcdq              object
datacqtr            object
datafqtr            object
rdq                 object
atq                float64
ceqq               float64
cheq               float64
cogsq              float64
cshfdq             float64
cshiq              float64
cshopq             float64
cshoq              float64
dlcq               float64
dl

In [41]:
compustat_names = \
    {# Identifiers
     'GVKEY': 'Gvkey',
     'LPERMNO': 'Permno',
     'LPERMCO': 'Permco',
     'fyearq': 'Fiscal Year',
     'fqtr': 'Fiscal Quarter',
     'conm': 'Company Name',
     'curcdq': 'Currency',
     'rdq': 'Report Date',     
     'naics': 'NAICS Sector Code',
     'exchg': 'Exchange Code',
     'datadate': 'datadate',
     
     # Balance Sheet
     'atq': 'Assets, Total',
     'ceqq': 'Common Equity, Total',
     'seqq': 'Shareholder Equity, Total',
     'pstkrq': 'Preferred Equity, Total',
     'ltq': 'Liabilities, Total',
     'txditcq': 'Deferred Tax Assets',
     'dlttq': 'Long Term Debt',
     'dlcq': 'Short Term Debt',
     'cheq': 'Cash',
        
     # Income Statement
     'saleq': 'Sales',
     'cogsq': 'COGS',
     'xsgaq': 'SG&A',
     'dpq': 'Depreciation and Amortization',
     'oibdpq': 'EBITDA',
     'oiadpq': 'EBIT',
     'xintq': 'Interest Expense',
     'txpq': 'Taxes Payable',
     'niq': 'Net Income',
     'epsfiq': 'Diluted EPS, Raw',
     'epsfxq': 'Diluted EPS, Adjusted',
     
     # Cash Flow Statement
     'oancfy': 'Operating Cash Flow',
     'fincfy': 'Financing Activities',
     'dltisy': 'Long Term Debt, Gross Issuance',
     'dltry': 'Long Term Debt, Retired',
     'dvy': 'Cash Dividends',
     'capxy': 'Capex',
     'aqcy': 'M&A',
     'cshopq': 'Total Repurchased Shares',
     'prcraq': 'Repurchase Price',
        
     # Market Data
     'cshoq': 'Shares Outstanding (Compustat)',
     'prccq': 'Price (Compustat)',
     'cshfdq': 'Shares Outstanding for EPS'}
compustat = compustat.rename(index = str, columns = compustat_names)
compustat = compustat[list(compustat_names.values())]

In [42]:
compustat = compustat.set_index(['Gvkey', 'datadate', 'Fiscal Year'])

In [43]:
# Convert the yearly variables
yearly_variables = sorted([compustat_names[x] for x in list(compustat_names.keys()) if x[-1] == 'y'])

In [44]:
def take_diffs(x):
    
    if x.name[1] == 2000:
        print(x.name)
        
    if all(np.isnan(x)):
        return x
    
    x = x.fillna(0)
    ret = x.diff()
    ret[0] = x[0]
    return ret


apple = compustat.xs(1166, level = 'Gvkey', drop_level = False)
orchard = apple.groupby(by = ['Gvkey', 'Fiscal Year'])

for v in yearly_variables:
    print(orchard[v].transform(take_diffs))

(1166, 2000)
Gvkey  datadate    Fiscal Year
1166   1981-06-30  1981              NaN
       1981-09-30  1981              NaN
       1981-12-31  1981              NaN
       1982-03-31  1982              NaN
       1982-06-30  1982              NaN
       1982-09-30  1982              NaN
       1982-12-31  1982              NaN
       1983-03-31  1983              NaN
       1983-06-30  1983              NaN
       1983-09-30  1983              NaN
       1983-12-31  1983              NaN
       1984-03-31  1984            0.000
       1984-06-30  1984            0.000
       1984-09-30  1984            0.000
       1984-12-31  1984           14.363
       1985-03-31  1985            0.000
       1985-06-30  1985            0.000
       1985-09-30  1985            0.000
       1985-12-31  1985           15.990
       1986-03-31  1986            0.000
       1986-06-30  1986            0.000
       1986-09-30  1986            0.000
       1986-12-31  1986            8.907
       1987-0

Gvkey  datadate    Fiscal Year
1166   1981-06-30  1981              NaN
       1981-09-30  1981              NaN
       1981-12-31  1981              NaN
       1982-03-31  1982              NaN
       1982-06-30  1982              NaN
       1982-09-30  1982              NaN
       1982-12-31  1982              NaN
       1983-03-31  1983              NaN
       1983-06-30  1983              NaN
       1983-09-30  1983              NaN
       1983-12-31  1983              NaN
       1984-03-31  1984            0.000
       1984-06-30  1984            0.000
       1984-09-30  1984            0.000
       1984-12-31  1984            0.000
       1985-03-31  1985            0.000
       1985-06-30  1985            0.000
       1985-09-30  1985            0.000
       1985-12-31  1985            0.000
       1986-03-31  1986            0.000
       1986-06-30  1986            0.000
       1986-09-30  1986            0.000
       1986-12-31  1986            0.000
       1987-03-31  1987   

In [45]:
cashflow_groups = compustat[yearly_variables].groupby(['Gvkey', 'Fiscal Year'])

In [46]:
from multiprocessing import Pool
import time

p = Pool(4)

def cash_flow(variable):
    print(variable)
    return cashflow_groups[variable].transform(take_diffs)

In [47]:
with Pool(4) as p:
    new_dataframes = p.map(cash_flow, yearly_variables)

Cash Dividends
Financing Activities
Long Term Debt, Gross Issuance
Capex
(1004, 2000)
(1004, 2000)
(1004, 2000)
(1004, 2000)
(1013, 2000)
(1013, 2000)
(1013, 2000)
(1013, 2000)
(1034, 2000)
(1036, 2000)
(1034, 2000)
(1034, 2000)
(1034, 2000)
(1038, 2000)
(1036, 2000)
(1036, 2000)
(1036, 2000)
(1038, 2000)
(1038, 2000)
(1038, 2000)
(1043, 2000)
(1045, 2000)
(1043, 2000)
(1043, 2000)
(1043, 2000)
(1045, 2000)
(1050, 2000)
(1045, 2000)
(1045, 2000)
(1050, 2000)
(1050, 2000)
(1050, 2000)
(1056, 2000)
(1062, 2000)
(1056, 2000)
(1056, 2000)
(1056, 2000)
(1072, 2000)
(1062, 2000)
(1062, 2000)
(1062, 2000)
(1075, 2000)
(1072, 2000)
(1072, 2000)
(1072, 2000)
(1076, 2000)
(1075, 2000)
(1075, 2000)
(1075, 2000)
(1078, 2000)
(1076, 2000)
(1076, 2000)
(1076, 2000)
(1081, 2000)
(1082, 2000)
(1078, 2000)
(1078, 2000)
(1078, 2000)
(1094, 2000)
(1081, 2000)
(1081, 2000)
(1081, 2000)
(1082, 2000)
(1095, 2000)
(1082, 2000)
(1082, 2000)
(1097, 2000)
(1094, 2000)
(1094, 2000)
(1094, 2000)
(1099, 2000)
(109

In [48]:
for d in new_dataframes:
    compustat[d.name] = d

In [49]:
compustat.head()

In [50]:
# Make some variables
compustat['Market Cap (Compustat)'] = compustat['Price (Compustat)'] * compustat['Shares Outstanding (Compustat)'] / 1e3

In [56]:
compustat.columns

Index(['Permno', 'Permco', 'Fiscal Quarter', 'Company Name', 'Currency',
       'Report Date', 'NAICS Sector Code', 'Exchange Code', 'Assets, Total',
       'Common Equity, Total', 'Shareholder Equity, Total',
       'Preferred Equity, Total', 'Liabilities, Total', 'Deferred Tax Assets',
       'Long Term Debt', 'Short Term Debt', 'Cash', 'Sales', 'COGS', 'SG&A',
       'Depreciation and Amortization', 'EBITDA', 'EBIT', 'Interest Expense',
       'Taxes Payable', 'Net Income', 'Diluted EPS, Raw',
       'Diluted EPS, Adjusted', 'Operating Cash Flow', 'Financing Activities',
       'Long Term Debt, Gross Issuance', 'Long Term Debt, Retired',
       'Cash Dividends', 'Capex', 'M&A', 'Total Repurchased Shares',
       'Repurchase Price', 'Shares Outstanding (Compustat)',
       'Price (Compustat)', 'Shares Outstanding for EPS',
       'Market Cap (Compustat)'],
      dtype='object')

In [60]:
compustat = compustat.reset_index().set_index(['Permco', 'datadate'])
x = compustat.xs(47, level = 'Permco', drop_level = False).xs('1988-03-31', level = 'datadate', drop_level = False)

In [61]:
x.apply(pd.Series.nunique, axis = 0)

Gvkey                             1
Fiscal Year                       1
Permno                            2
Fiscal Quarter                    1
Company Name                      1
Currency                          1
Report Date                       1
NAICS Sector Code                 1
Exchange Code                     1
Assets, Total                     1
Common Equity, Total              1
Shareholder Equity, Total         1
Preferred Equity, Total           1
Liabilities, Total                1
Deferred Tax Assets               1
Long Term Debt                    1
Short Term Debt                   1
Cash                              1
Sales                             1
COGS                              1
SG&A                              1
Depreciation and Amortization     1
EBITDA                            1
EBIT                              1
Interest Expense                  1
Taxes Payable                     1
Net Income                        1
Diluted EPS, Raw            

In [ ]:
x

In [ ]:
compustat.xs(47, level = 'Permco', drop_level = False).xs('1988-03-31', level = 'datadate', drop_level = False)

In [ ]:
store.put('compustat_only', compustat)

## Combine the two dataframes

In [ ]:
compustat = compustat.reset_index().set_index(['Permco', 'datadate'])
crsp = crsp.reset_index().set_index(['Permco', 'datadate'])

In [ ]:
merged_datasets = crsp.join(compustat, how = 'left', lsuffix = '.crsp', rsuffix = '.comp')

In [ ]:
compustat.xs(7, level = 'Permco', drop_level = False)

In [ ]:
crsp.loc[crsp['Company Name'] == 'APPLE INC']

In [ ]:
merged_datasets.shape

In [ ]:
crsp.shape

In [ ]:
crsp.xs(540, level = 'Permco')

In [ ]:
merged_datasets.xs(7, level = 'Permco')